In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import imgaug as ia
from imgaug import augmenters as iaa
import tensorflow as tf

from CA1_Yolo_Kenneth import *
from tensorflow.keras.optimizers import Adam
import time

In [6]:
img_dir = "F://Users//Kenneth//Documents//GitHub//ISS-VSE-2019-09-23-IS1FT-CA1//data//image//"
anno_dir = "F://Users//Kenneth//Documents//GitHub//ISS-VSE-2019-09-23-IS1FT-CA1//data//annotation//"

In [7]:
all_anno, labels = read_annotation_files(img_dir, anno_dir)

In [8]:
anchor_boxes = generateAnchorBoxes(all_anno, labels)

In [9]:
anchor_boxes

[8, 11, 14, 18, 26, 26, 36, 45, 46, 31, 48, 82, 68, 47, 85, 76, 90, 120]

In [10]:
train_size=0.8
train_valid_split = int(0.8 * len(all_anno))
np.random.shuffle(all_anno)
train_anno = all_anno[:train_valid_split]
valid_anno = all_anno[train_valid_split:]
max_boxes = max([len(anno['object']) for anno in (train_anno + valid_anno)])

In [11]:
len(train_anno)

612

In [12]:
train_generator = DataGenerator(
    annotations=train_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=2,
    width=416,
    height=416,
    shuffle=True)

In [13]:
len(train_generator)

306

In [14]:
valid_generator = DataGenerator(
    annotations=valid_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=2,
    width=416,
    height=416,
    shuffle=True)

In [15]:
len(valid_generator)

76

In [16]:
train_model, infer_mode = YoloV3(
    numcls=len(labels),
    anchors=anchor_boxes,
    max_grid=[416, 416],
    batch_size=1,
    threshold=0.5,
    max_boxes=max_boxes)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [17]:
infer_mode.build((416,416))
infer_mode.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, None, None, 3 128         conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, None, None, 3 0           batch_normalization_v1[0][0]     
__________________________________________________________________________________________________
zero_paddi

In [18]:
def dummy_loss(y_true, y_pred):
    return tf.sqrt(tf.reduce_sum(y_pred))

In [19]:
opt = Adam(lr=1e-4, clipnorm=0.001)
train_model.compile(loss=dummy_loss, optimizer=opt)

In [20]:
from tensorflow.keras.callbacks import TensorBoard
NAME = "YoloV3-training-{}".format(int(time.time()))

filepath='F:\\Users\\Kenneth Goh\\Documents\\GitHub\\ISS-VSE-2019-09-23-IS1FT-CA1-Part1\\code\\training_1\\cp.chkpt'# add filepath
callback = create_callbacks(filepath)

tensorboardlog = ".\\training_1\\log\\"
tensorboard = TensorBoard(log_dir=tensorboardlog, histogram_freq=1)

In [ ]:
%%time
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.save(sess,'./training_1/cp.ckpt')
    tf.train.write_graph(sess.graph.as_graph_def(), '.', 'tensorflowModel.pbtxt', as_text=True)
    history = train_model.fit_generator(
        generator=train_generator,
        validation_data=valid_generator,
        verbose=1,
        steps_per_epoch=len(train_generator),
        epochs=3, #for testing
        callbacks=[tensorboard]
    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
  2/306 [..............................] - ETA: 7:31:32 - loss: 140.6103 - yolo_loss_layer_loss: 22.8239 - yolo_loss_layer_1_loss: 37.2294 - yolo_loss_layer_2_loss: 78.2291

In [19]:
%load_ext tensorboard #localhost:6006 will work too
%tensorboard --logdir=training_1/log

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 148.

In [20]:
!kill 28452 #have to close tensorboard if you want to relaunch it.

'kill' is not recognized as an internal or external command,
operable program or batch file.


In [22]:
meta_path = './training_1/cp.ckpt.meta' # Your .meta file
output_node_names = [node.op.name for node in train_model.outputs]    # Output nodes
print(output_node_names)

['yolo_loss_layer/add_7', 'yolo_loss_layer_1/add_7', 'yolo_loss_layer_2/add_7']


In [23]:
with tf.Session() as sess:
    # Restore the graph
    saver = tf.train.import_meta_graph(meta_path)

    # Load weights
    saver.restore(sess,tf.train.latest_checkpoint('./training_1/'))

    # Freeze the graph
    frozen_graph_def = tf.graph_util.convert_variables_to_constants(
        sess,
        sess.graph_def,
        output_node_names)

    # Save the frozen graph
    with open('output_graph2.pb', 'wb') as f:
        f.write(frozen_graph_def.SerializeToString())

INFO:tensorflow:Restoring parameters from ./training_1/cp.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 366 variables.
INFO:tensorflow:Converted 366 variables to const ops.


In [24]:
from tensorflow.python.tools import optimize_for_inference_lib

inputGraph = tf.GraphDef()
with tf.gfile.Open('output_graph2.pb', "rb") as f:
    data2read = f.read()
    inputGraph.ParseFromString(data2read)

inputTensor = [node.name for node in inputGraph.node if node.op=='Placeholder'] #[n.name + '=>' +  n.op for n in inputGraph.node if n.op in ( 'Softmax','Placeholder')]
    
outputGraph = optimize_for_inference_lib.optimize_for_inference(
              inputGraph,
              inputTensor, # an array of the input node(s)
              output_node_names, # an array of output nodes
              tf.int32.as_datatype_enum)

# Save the optimized graph'test.pb'
f = tf.gfile.FastGFile('OptimizedGraph.pb', "w")
f.write(outputGraph.SerializeToString()) 

Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
Instructions for updating:
Use tf.gfile.GFile.


In [25]:
import cv2
 
# Load a model imported from Tensorflow
tensorflowNet = cv2.dnn.readNetFromTensorflow('OptimizedGraph.pb', 'tensorflowModel.pbtxt')

error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\dnn\src\tensorflow\tf_io.cpp:54: error: (-2:Unspecified error) FAILED: ReadProtoFromTextFile(param_file, param). Failed to parse GraphDef file: tensorflowModel.pbtxt in function 'cv::dnn::ReadTFNetParamsFromTextFileOrDie'
